In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopy.distance
import re

# Load the data
path = '/Users/rfd/vigilo-notebooks-bapav/vigilo2024/data_collection'
df_incidents = pd.read_csv(f'{path}/df_incidents.csv')
df_poi = pd.read_csv(f'{path}/df_poi.csv')
df_incidents.head()

,token,latitude,longitude,address,comment,explanation,time,categorie,cityname
0,AA1CA063,43.606022,3.873078,"Cours Gambetta, Montpellier",Roue dans les Rails du tram,La personne venait d’une rue perpendiculaire e...,2024-05-16 11:55:00,9,Montpellier
1,AF00A88E,43.617920,3.882070,"Rue de la Cavalerie, Montpellier",stationnement gênant récurrent,stationnement gênant sur le trottoir systémati...,2024-05-15 19:03:00,2,Montpellier
2,B9B3C9BA,43.596166,3.897689,"Rue des Acconiers, Montpellier",Risque de chute,Poteau anti-stationnement en travers de la piste,2024-05-11 17:32:00,2,Montpellier
3,1441837B,43.598695,3.904834,"Avenue Nina Simone, Montpellier",Road to nowhere,NaN,2024-05-08 15:34:00,3,Montpellier
4,CA159445,43.696450,3.810200,"Rue de Coulondres, Saint-Gély-du-Fesc",Véhicule stationné sur piste cyclable,NaN,2024-05-08 09:29:00,2,Saint-Gély-du-Fesc


In [62]:
# Function to preprocess text
def preprocess_text(text):
    # Convert to lowercase
    text = str(text).lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Apply the preprocessing function to the comment column
df_incidents['comment_processed'] = df_incidents['comment'].apply(preprocess_text)
# Apply the preprocessing function to the explanation column
df_incidents['explanation_processed'] = df_incidents['explanation'].apply(preprocess_text)


In [63]:
keywords = [
    'voiture',
    'stationnée',
    'trottoir',
    'piste cyclable',
    'bloque',
    'double file',
    'gênant',
    'obstrue',
    'illégal',
    'voie cyclable',
    'bande cyclable',
    'accès',
    'abusif',
    'circulation',
    'sauvage',
    'voie verte',
    'passage',
    'route cyclable',
    'chemin',
    'interdit',
    'gêne',
    'sentier',
    'réservée',
    'piétonne',
    'cycliste',
    'voiture sur piste',
    'stationnement gênant',
    'stationnement sur trottoir',
    'stationnement sur piste',
    'bloque accès',
    'voiture bloque',
    'voiture obstrue',
    'stationnement abusif',
    'gêne passage',
    'gêne cycliste',
    'stationnement sauvage',
    'stationnement interdit',
    'voie réservée',
    'piste vélo',
    'voie vélo',
    'bande vélo',
    'voiture sur chemin',
    'bloque route',
    'empêche passage',
    'chemin cycliste',
    'sentier vélo',
    'voiture gênante',
    'voie piétonne'
]


def verify_keywords(phrase):
    count = sum(1 for word in keywords if word in phrase)
    return count >= 3

# Apply the keyword search function to the processed text columns and create a new column
df_incidents['contains_keyword'] = df_incidents['comment_processed'].apply(verify_keywords) | df_incidents['explanation_processed'].apply(verify_keywords)
df = df_incidents[df_incidents['contains_keyword']==True].drop(columns=['comment_processed', 'explanation_processed'])
df.head()


,token,latitude,longitude,address,comment,explanation,time,categorie,cityname,contains_keyword
1,AF00A88E,43.617920,3.882070,"Rue de la Cavalerie, Montpellier",stationnement gênant récurrent,stationnement gênant sur le trottoir systémati...,2024-05-15 19:03:00,2,Montpellier,True
42,63812A40,43.610533,3.849704,"Avenue de Lodève, Montpellier",bordure saillante de 2cm invisible dangereuse,"voie cyclable avenue de lodeve, en bout de lig...",2024-02-09 15:30:00,3,Montpellier,True
95,7DA37B07,43.635835,3.925538,"Route de Nîmes, Castelnau-le-Lez",Bloque trottoir et piste cyclable,NaN,2023-11-14 12:37:00,2,Castelnau-le-Lez,True
128,2035993A,43.598541,3.875804,"Avenue de Maurin, Montpellier",Camion de livraison en arrêt sur la piste cyclabl,"Régulièrement, aujourd’hui ça fait 2 jours con...",2023-10-05 07:03:00,2,Montpellier,True
179,7D1F714E,43.609550,3.881651,"Avenue Frédéric Mistral, Montpellier",chaque jour en infraction,R417-10 : stationnement interdit en aire piétonne,2023-09-13 06:00:00,2,Montpellier,True


In [64]:
BUFFER = 300  # Buffer in meters

# Function to calculate the distance between two points
def calculate_distance(lat1, lon1, lat2, lon2):
    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)
    return geopy.distance.distance(coords_1, coords_2).meters

# Initialize a column for storing nearby POIs
df['POI à proximité'] = None

#Ajouter une colonne avec le nombre de signalement à proximité et par catégorie de signalement pour chaque point d'intérêt
for poi_id, poi in df_poi.iterrows():
    # Initialize a column for the count of nearby issues
    df_poi.loc[poi_id, 'Nombre de signalements à proximité'] = 0

    for issue_id, issue in df.iterrows():
        distance = calculate_distance(poi['latitude'], poi['longitude'], issue['latitude'], issue['longitude'])
        if distance <= BUFFER:
            df_poi.loc[poi_id, 'Nombre de signalements à proximité'] += 1

df_poi

,Unnamed: 0,name,latitude,longitude,Nombre de signalements à proximité
0,252279166,École primaire privée Saint-François-Régis,43.608725,3.871593,0.0
1,585171936,Crèche La Gerbe,43.605247,3.870653,4.0
2,585173372,Crèche collective municipale Antigone,43.607227,3.884581,0.0
3,585182991,Crèche collective municipale Les Sept Nains,43.614204,3.879547,7.0
4,585199240,Crèche associative La Bambinerie,43.611569,3.855149,2.0
5,595252160,Jardin d'enfant municipal Les Boutons d'Or,43.622028,3.888476,0.0
6,595256929,Crèche associative Accroche Cœur,43.617687,3.882115,3.0
7,598191190,Jardin d'enfant privé Le Petit Home,43.615837,3.862836,3.0
8,600742273,École primaire Notre-Dame de Bonne-Nouvelle,43.607060,3.881109,4.0
9,607355557,Crèche municipale Edelweiss,43.603467,3.880709,2.0
